In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#%cd /content/drive/MyDrive/

In [3]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
#import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers


# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

print("OK\n")

OK



In [6]:
app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')

In [7]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()

In [9]:
app_train.isnull().sum()

SK_ID_CURR                    0
TARGET                        0
NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
                           ... 
credit_annuity_ratio         12
DAYS_EMPLOYED_ANOM            0
credit_goods_price_ratio    278
credit_downpayment          278
AGE_INT                       0
Length: 93, dtype: int64

In [28]:
data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_train = X_train.select_dtypes(include=[int, float]).fillna(-999)
X_test = app_test.copy()
X_test = X_test.select_dtypes(include=[int, float]).fillna(-999)
len(X_test)

48744

In [30]:
y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)


for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    tr_x = X_train.loc[train_index, :]
    va_x = X_train.loc[valid_index, :]
    tr_y = y_train[train_index]
    va_y = y_train[valid_index]
    
    scaler = StandardScaler()
    tr_x = scaler.fit_transform(tr_x)
    va_x = scaler.transform(va_x)
    test_x = scaler.transform(X_test)
    
    model = Sequential()
    model.add(Dense(
        256
        ,activation = 'relu'
        ,input_shape=(X_train.shape[1],)
    ))
    
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
   
    model.compile(loss='binary_crossentropy',
                 optimizer='adam', metrics=['accuracy'])
    
    # 学習の実行
    batch_size = 128
    epochs = 10
    history = model.fit(tr_x, tr_y,
                       batch_size=batch_size, epochs=epochs,
                       verbose=1, validation_data=(va_x, va_y))
    
    va_pred = model.predict(va_x)
    y_pred = model.predict(test_x)
    score = log_loss(va_y, va_pred, eps=1e-7)
    print(f'logloss:{score:.4f}')
        
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

Epoch 1/10
1922/1922 [==============================] - 5s 2ms/step - loss: 0.2743 - accuracy: 0.9185 - val_loss: 0.2620 - val_accuracy: 0.9211
Epoch 2/10
1922/1922 [==============================] - 4s 2ms/step - loss: 0.2701 - accuracy: 0.9188 - val_loss: 0.2654 - val_accuracy: 0.9211
Epoch 3/10
1922/1922 [==============================] - 4s 2ms/step - loss: 0.2686 - accuracy: 0.9188 - val_loss: 0.2611 - val_accuracy: 0.9211
Epoch 4/10
1922/1922 [==============================] - 4s 2ms/step - loss: 0.2676 - accuracy: 0.9188 - val_loss: 0.2601 - val_accuracy: 0.9211
Epoch 5/10
1922/1922 [==============================] - 4s 2ms/step - loss: 0.2671 - accuracy: 0.9188 - val_loss: 0.2612 - val_accuracy: 0.9211
Epoch 6/10
1922/1922 [==============================] - 4s 2ms/step - loss: 0.2666 - accuracy: 0.9188 - val_loss: 0.2610 - val_accuracy: 0.9211
Epoch 7/10
1922/1922 [==============================] - 4s 2ms/step - loss: 0.2659 - accuracy: 0.9188 - val_loss: 0.2612 - val_accuracy:

In [31]:
scores

[0.26161503008461384,
 0.2646827786407651,
 0.26477791108017457,
 0.2664224799956247,
 0.2670937117017784]

In [32]:
y_preds

[array([[0.03825499],
        [0.01464197],
        [0.01216851],
        ...,
        [0.0709767 ],
        [0.03375876],
        [0.098987  ]], dtype=float32),
 array([[0.02754524],
        [0.00669081],
        [0.02876271],
        ...,
        [0.04949496],
        [0.01737387],
        [0.13064407]], dtype=float32),
 array([[0.05532917],
        [0.00498434],
        [0.03595953],
        ...,
        [0.07397807],
        [0.04071283],
        [0.12405236]], dtype=float32),
 array([[0.06279404],
        [0.00955997],
        [0.0456911 ],
        ...,
        [0.05851615],
        [0.05543239],
        [0.14372446]], dtype=float32),
 array([[0.05589843],
        [0.0106366 ],
        [0.0436075 ],
        ...,
        [0.079992  ],
        [0.04229377],
        [0.13690862]], dtype=float32)]

In [33]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [34]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [35]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [36]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [37]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [38]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [39]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.038255,0.027545,0.055329,0.062794,0.055898
1,0.014642,0.006691,0.004984,0.009560,0.010637
2,0.012169,0.028763,0.035960,0.045691,0.043608
3,0.060577,0.039731,0.080779,0.051872,0.050830
4,0.079823,0.084593,0.086336,0.116570,0.108352


In [40]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.820734,0.892509,0.866980,0.824101
submit_all_01,0.820734,1.000000,0.880867,0.932365,0.943032
submit_all_02,0.892509,0.880867,1.000000,0.899847,0.894627
submit_all_03,0.866980,0.932365,0.899847,1.000000,0.941097
submit_all_04,0.824101,0.943032,0.894627,0.941097,1.000000


In [41]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.047964
1,100005,0.009303
2,100013,0.033238
3,100028,0.056758
4,100038,0.095135


In [42]:
len(submit_all)

48744

In [43]:
len(app_test)

48744

In [44]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/hcr_app_nn_res_01.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.009593
1,100005,0.001861
2,100013,0.006648
3,100028,0.011352
4,100038,0.019027
